In [ ]:
import urllib.request,sys,time
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re

linkset=set()
categorylist=["https://www.aninews.in/category/national/","https://www.aninews.in/category/world/","https://www.aninews.in/category/tech/",'https://www.aninews.in/category/health/']
nooflinks=0
for url in categorylist:
    if nooflinks>4000:
      break
    print('processing category :', url)
    
    try:
        page=requests.get(url)                            
    
    except Exception as e:                                   
        error_type, error_obj, error_info = sys.exc_info()      
        print ('ERROR FOR LINK:',url)                          
        print (error_type, 'Line:', error_info.tb_lineno)     
        continue                                              
    time.sleep(2)   
    soup=BeautifulSoup(page.text,'html.parser')
    category=set()
    for a in soup.find_all('a', href=True):
      matchObj = re.match( r'/news', a['href'], re.M|re.I)
      if matchObj:
        finallink="https://www.aninews.in"+str(a['href'])
        linkset.add(finallink)
      else:
        matchObj = re.match( r'/category', a['href'], re.M|re.I)
        if matchObj:
          finalcategory="https://www.aninews.in/"+str(a['href'])
          category.add(finalcategory)
    for c in category:
      if c not in categorylist:
        categorylist.append(c)
    nooflinks=len(linkset)
print(len(linkset))

processing category : https://www.aninews.in/category/national/
processing category : https://www.aninews.in/category/world/
processing category : https://www.aninews.in/category/tech/
processing category : https://www.aninews.in/category/health/
processing category : https://www.aninews.in//category/world/
processing category : https://www.aninews.in//category/lifestyle/parenting/
processing category : https://www.aninews.in//category/environment/
processing category : https://www.aninews.in//category/world/us/
processing category : https://www.aninews.in//category/world/middle-east/
processing category : https://www.aninews.in//category/national/
processing category : https://www.aninews.in//category/sports/others/
processing category : https://www.aninews.in//category/national/page/4
processing category : https://www.aninews.in//category/lifestyle/travel/
processing category : https://www.aninews.in//category/tech/computers/
processing category : https://www.aninews.in//category/nat

In [ ]:
#All the scraped links
print(linkset)

{'https://www.aninews.in/news/lifestyle/parenting/covid-19-cannot-be-transmitted-through-breast-milk-study-suggests20200823220504', 'https://www.aninews.in/news/world/europe/uk-records-over-23000-new-coronavirus-cases-total-count-at-85401020201025020801', 'https://www.aninews.in/news/lifestyle/fashion/la-fashion-and-beauty-retailers-take-steps-to-contain-coronavirus-spread20200314105524', 'https://www.aninews.in/news/lifestyle/parenting/premature-borns-more-likely-to-be-placed-outside-home-study20191228191317', 'https://www.aninews.in/news/entertainment/out-of-box/kim-kardashian-stunningly-poses-on-the-beach-in-bronze-bikini20200828225320', 'https://www.aninews.in/news/world/middle-east/syria-shoots-down-several-targets-in-hamas-airspace20200624071943', 'https://www.aninews.in/news/lifestyle/culture/google-honours-activist-kamini-roy-with-doodle-on-her-155th-birthday20191012084314', 'https://www.aninews.in/news/national/general-news/heavy-rains-likely-over-northeast-today20201024090639

In [ ]:
frame=[]
filename="FinalANINews.csv"
f=open(filename,"w", encoding = 'utf-8')
headers="Article Id,Heading,Link,Body,Time,Tags\n"
f.write(headers)
linkset=list(linkset)
for i in range(410,2000):
  try:
    url=linkset[i]
    page=requests.get(url)                             # this might throw an exception if something goes wrong.
    time.sleep(2)   
    soup=BeautifulSoup(page.text,'html.parser')   
    Heading= soup.find("h1",attrs={'class':'title'}).text.strip()
    Body=soup.find("div",attrs={'itemprop':'articleBody'}).text.strip()
    Time=soup.find("span",attrs={'class':'time-red'}).text.strip()
    Tags=soup.find("div",attrs={'class':'social-tags'}).text.strip()
    Tags=re.sub('Tags', '', Tags)
    Tags=re.sub('\n', '', Tags) #this one is for csv file in a string format
    Tagss=Tags.split(" ")       #this one is for dataframe
    frame.append((i,Heading,url,Body,Time,Tagss))
    f.write(str(i)+","+Heading.replace(",","^")+","+url+","+Body.replace(","," ")+","+Time.replace(",","")+","+Tags+"\n")
  except Exception as e:  
    continue
f.close()
data=pd.DataFrame(frame, columns=['ArticleID','Heading','Link','Body','Time','Tags'])
data.head()

,ArticleID,Heading,Link,Body,Time,Tags
0,410,Roger Federer reveals how many pair of shoes h...,https://www.aninews.in/news/lifestyle/fashion/...,"Washington DC [USA], Dec 3 (ANI): Roger Federe...","Dec 03, 2019 01:49","[Roger, Federer, shoes, On, Nike, Swiss]"
1,411,"Here's Tara Sutaria, all 'ready to shake up Bo...",https://www.aninews.in/news/lifestyle/fashion/...,"New Delhi (India), Aug 9 (ANI): Dolled up as a...","Aug 09, 2019 09:22","[Tara, Sutaria, Ahan, Shetty, Milan, Luthria, ..."
2,412,Emmys 2019: Celebrities arrive in style for th...,https://www.aninews.in/news/lifestyle/fashion/...,"Los Angeles [USA], Sept 23 (ANI): The biggest ...","Sep 23, 2019 07:11","[Emmys, 2019, Emmy, Awards, best, dressed, cel..."
3,414,Google Assistant Snapshot gets new features,https://www.aninews.in/news/tech/mobile/google...,"California [US], August 28 (ANI): Google Assis...","Aug 28, 2020 15:10","[Google, Assistant, Google, Assistant, Snapsho..."
4,415,Danish students renovate Tibetan monument in I...,https://www.aninews.in/news/national/features/...,"Dharamsala, (Himachal Pradesh) [India] Apr 8 (...","Apr 08, 2018 19:14","[Denmark, India, Dharamsala, Himachal, Pradesh..."


In [ ]:
import pandas as pd
import numpy as np
userdata=pd.read_csv('userdata.csv')
userdata.head()

,Unnamed: 0,user_id,session_id,article_id,clicked_or_not,time_spent,rating_article
0,0,1,1,915,1,81.194235,3
1,1,1,1,1306,0,0.000000,0
2,2,1,1,629,0,0.000000,0
3,3,1,1,512,0,0.000000,0
4,4,1,1,1830,0,0.000000,0


In [ ]:
newsdata=pd.read_csv('aninewsdata.csv')
newsdata.head()

,Article Id,Heading,Link,Body,Time,Tags
0,0,COVID-19 cannot be transmitted through breast ...,https://www.aninews.in/news/lifestyle/parentin...,California [US] August 23 (ANI): Breast milk ...,Aug 23 2020 22:05,COVID-19 Transmission Breast milk Infant
1,1,UK records over 23^000 new coronavirus cases^ ...,https://www.aninews.in/news/world/europe/uk-re...,London [UK] October 25 (ANI/Xinhua): Another ...,Oct 25 2020 02:08,UK coronavirus United Kingdom COVID-19
2,2,LA fashion and beauty retailers take steps to ...,https://www.aninews.in/news/lifestyle/fashion/...,California [USA] Mar 14 (ANI): As the coronav...,Mar 14 2020 10:55,Los Angeles fashion stores Beauty retailers on...
3,3,Premature borns more likely to be placed outsi...,https://www.aninews.in/news/lifestyle/parentin...,Washington D.C. [USA] Dec 28 (ANI): A study r...,Dec 28 2019 19:15,Parenting Premature babies Study Child welfare...
4,4,Kim Kardashian stunningly poses on the beach i...,https://www.aninews.in/news/entertainment/out-...,Washington [US] Aug 28 (ANI): American realit...,Aug 28 2020 22:55,Kim Kardashian Bronze Bikini Photo shoot Malib...


# News Data Preprocessing



In [ ]:
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
def onlytext(body):
  body = re.sub(r'[^\w\s]', ' ', body)  #removing special characters
  body = re.sub(' +',' ',body) #double spaces
  numbers = '[0-9]'
  body ="".join([re.sub(numbers, '', i) for i in body]) #numbers
  body=body.split()
  body=body[4:]     #removes some arbitary data
  stop_words = set(stopwords.words('english'))
  wordnet_lemmatizer = WordNetLemmatizer()
  body = [wordnet_lemmatizer.lemmatize(word,pos='v') for word in body]
  body = [word for word in body if not word in (stop_words)]
  body= (" ".join(body)).lower()
  return body
newsdata['cleaned'] = newsdata['Body'].apply(func = onlytext)
#print(onlytext("California [US] August 23 (ANI):This is America 1991 !!!! "))

In [ ]:
newsdata.head()

,Article Id,Heading,Link,Body,Time,Tags,cleaned
0,0,COVID-19 cannot be transmitted through breast ...,https://www.aninews.in/news/lifestyle/parentin...,California [US] August 23 (ANI): Breast milk ...,Aug 23 2020 22:05,COVID-19 Transmission Breast milk Infant,breast milk cannot transmit covid virus infant...
1,1,UK records over 23^000 new coronavirus cases^ ...,https://www.aninews.in/news/world/europe/uk-re...,London [UK] October 25 (ANI/Xinhua): Another ...,Oct 25 2020 02:08,UK coronavirus United Kingdom COVID-19,xinhua another people britain test positive co...
2,2,LA fashion and beauty retailers take steps to ...,https://www.aninews.in/news/lifestyle/fashion/...,California [USA] Mar 14 (ANI): As the coronav...,Mar 14 2020 10:55,Los Angeles fashion stores Beauty retailers on...,as coronavirus pandemic spread hollywood talen...
3,3,Premature borns more likely to be placed outsi...,https://www.aninews.in/news/lifestyle/parentin...,Washington D.C. [USA] Dec 28 (ANI): A study r...,Dec 28 2019 19:15,Parenting Premature babies Study Child welfare...,dec ani a study reveal children bear preterm l...
4,4,Kim Kardashian stunningly poses on the beach i...,https://www.aninews.in/news/entertainment/out-...,Washington [US] Aug 28 (ANI): American realit...,Aug 28 2020 22:55,Kim Kardashian Bronze Bikini Photo shoot Malib...,american reality tv star kim kardashian photog...


## Topic Modelling

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import gensim
from gensim.utils import simple_preprocess
def headingspreprocessing(body):
  body = re.sub(r'[^\w\s]', ' ', body)  #removing special characters
  body = re.sub(' +',' ',body) #remove double spaces
  numbers = '[0-9]'
  body ="".join([re.sub(numbers, '', i) for i in body]) #remove numbers
  body=body.lower().split()
  stop_words = set(stopwords.words('english'))
  wordnet_lemmatizer = WordNetLemmatizer()
  body = [wordnet_lemmatizer.lemmatize(word,pos='v') for word in body]
  body = [word for word in body if not word in (stop_words)]
  return body
processed_titles = newsdata['Heading'].apply(func =  headingspreprocessing)
processed_titles[:10]

0    [covid, cannot, transmit, breast, milk, study,...
1    [uk, record, new, coronavirus, case, total, co...
2    [la, fashion, beauty, retailers, take, step, c...
3    [premature, borns, likely, place, outside, hom...
4    [kim, kardashian, stunningly, pose, beach, bro...
5      [syria, shoot, several, target, hama, airspace]
6    [google, honour, activist, kamini, roy, doodle...
7              [heavy, rain, likely, northeast, today]
8     [magnitude, earthquake, hit, papua, new, guinea]
9                     [j, k, report, new, covid, case]
Name: Heading, dtype: object

# Word2vec

In [ ]:
from gensim.models import Word2Vec
w2v_model = Word2Vec(min_count=10,window=2,size=300,sample=6e-5,alpha=0.03, min_alpha=0.0007, 
                     negative=20,workers=4)
w2v_model.build_vocab(processed_titles, progress_per=1000)
w2v_model.train(processed_titles, total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)
w2v_model.init_sims(replace=True)
print(w2v_model.corpus_count)

1992


Word2vec Results

In [ ]:
print("We will try to find topics similar to COVID")
w2v_model.wv.most_similar(positive=["covid"])

We will try to find topics similar to COVID


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('study', 0.9981043934822083),
 ('us', 0.9966894388198853),
 ('may', 0.9962678551673889),
 ('say', 0.9962381720542908),
 ('health', 0.9961346387863159),
 ('day', 0.9960851669311523),
 ('high', 0.9955377578735352),
 ('help', 0.99500572681427),
 ('trump', 0.9949465990066528),
 ('take', 0.9948573708534241)]

In [ ]:
print("Lets see the similarity between covid and health")
w2v_model.wv.similarity("covid", 'health')

Lets see the similarity between covid and health


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


0.9961345

We see a similarity score of 0.99 which says indeed Covid is associated with our health.
# Content Based Recommender Using Cosine Similarity


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel


In [ ]:
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(newsdata['Body'])
tfidf_matrix.shape

(1992, 32964)

In [ ]:
similarity_matrix = linear_kernel(tfidf_matrix,tfidf_matrix)
similarity_matrix

array([[1.        , 0.12658503, 0.11932332, ..., 0.05666455, 0.1306308 ,
        0.10560039],
       [0.12658503, 1.        , 0.13902098, ..., 0.08790361, 0.1709212 ,
        0.11895245],
       [0.11932332, 0.13902098, 1.        , ..., 0.10021974, 0.13983013,
        0.13542708],
       ...,
       [0.05666455, 0.08790361, 0.10021974, ..., 1.        , 0.07841653,
        0.07173286],
       [0.1306308 , 0.1709212 , 0.13983013, ..., 0.07841653, 1.        ,
        0.15256487],
       [0.10560039, 0.11895245, 0.13542708, ..., 0.07173286, 0.15256487,
        1.        ]])

In [ ]:
mapping = pd.Series(newsdata.index,index = newsdata['Heading'])
def recommend_CS(previousread):
  newsindex= mapping[previousread]
  similarity_score = list(enumerate(similarity_matrix[newsindex]))
  similarity_score = sorted(similarity_score, key=lambda x: x[1], reverse=True) #desc order
  similarity_score = similarity_score[1:10]
  recommended_news = [i[0] for i in similarity_score]
  for i in recommended_news:
    print(newsdata['Time'].iloc[i],newsdata['Heading'].iloc[i]+"\t" + newsdata['Link'].iloc[i])

In [ ]:
recommend_CS("Premature borns more likely to be placed outside home: Study")

Oct 07 2020 08:01 Expanded newborn screening could save premature infants' lives^ finds study	https://www.aninews.in/news/health/expanded-newborn-screening-could-save-premature-infants-lives-finds-study20201007080112
Jan 02 2020 00:34 More screen time adversely affect development in children: Study	https://www.aninews.in/news/lifestyle/parenting/more-screen-time-adversely-affect-development-in-children-study20200102003403
Aug 07 2019 22:35 Parental stress during pregnancy linked to behavioural problems in toddlers	https://www.aninews.in/news/lifestyle/parenting/parental-stress-during-pregnancy-linked-to-behavioural-problems-in-toddlers20190807223109
Jun 11 2018 18:24 Child labour continues to leave its mark on society	https://www.aninews.in/news/national/features/child-labour-continues-to-leave-its-mark-on-society201806111747490001
Sep 04 2020 11:24 Study finds antibiotics affect breast milk microbiota in mothers of preterm infants	https://www.aninews.in/news/health/study-finds-antibio

In [ ]:
print(newsdata['Heading'].iloc[788])  #finding random heading

Samsung's 'Find My Mobile' feature to function with offline devices


In [ ]:
recommend_CS("Samsung's 'Find My Mobile' feature to function with offline devices")

Aug 19 2020 15:16 Netflix tests 'shuffle' button on the homepage	https://www.aninews.in/news/tech/others/netflix-tests-shuffle-button-on-the-homepage20200819151627
Sep 11 2020 22:55 Disney Plus is planning to roll out new party watch feature	https://www.aninews.in/news/tech/internet/disney-plus-is-planning-to-roll-out-new-party-watch-feature20200911225532
Jul 15 2020 13:22 Samsung Galaxy A series users can have Galaxy S20 features after software update	https://www.aninews.in/news/tech/mobile/samsung-galaxy-a-series-users-can-have-galaxy-s20-features-after-software-update20200715132228
Aug 18 2020 22:59 Samsung names devices that will get 'three generations' of Android updates	https://www.aninews.in/news/tech/mobile/samsung-names-devices-that-will-get-three-generations-of-android-updates20200818225903
Jul 29 2020 10:35 Spotify expands its Group Session feature	https://www.aninews.in/news/tech/internet/spotify-expands-its-group-session-feature20200729103446
Oct 16 2020 23:51 Samsung smar

# Conclusion: 


1.   We see when we use cosine similarity recommender system is performing well as it gives us similar articles to our liking.

1.  We have to acknowledge the fact that currently the dataset is small so computation of similarity matrix is not a problem but as the dataset grows this becomes a problem so we can LSH as it realizes efficiencies in memory and number of computations performed.
2. But the suggestion lacks diversity( we can solve this collaborative filtering) 




